In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
# create a datset of 2D signals with noise
n_samples = 10_000
n_sensors = 4
n_measurements = 5
n_dimensions = 2
width = 1000

noise_range = 10
noise_bearing = 0.1

n_output_positions = 10
noise_output = 10.0

with torch.no_grad():
    targets = width / 2 * torch.rand(n_samples, n_dimensions) - width / 4
    sensors = width / 2 * torch.rand(n_samples, n_sensors, n_dimensions) / 2

    # relative position of targets to sensors
    dx = targets[:, None] - sensors
    dx = dx[..., None, :]
    # add noise in polar coordinates
    measurements_range = (
        torch.norm(dx, dim=-1)
        + torch.randn(n_samples, n_sensors, n_measurements) * noise_range
    )
    measurements_bearing = (
        torch.atan2(dx[..., 1], dx[..., 0])
        + torch.randn(n_samples, n_sensors, n_measurements) * noise_bearing
    )
    # convert back to cartesian coordinates
    measurements = sensors[..., None, :] + torch.stack(
        [
            measurements_range * torch.cos(measurements_bearing),
            measurements_range * torch.sin(measurements_bearing),
        ],
        dim=-1,
    )
    # combine the n_sensors and n_measurements dimensions
    measurements = measurements.reshape(
        n_samples, n_sensors * n_measurements, n_dimensions
    )
    targets = targets[:, None, None, :]
    measurements = measurements[:, None, :, :]

    # add noise to the measurements, to get the sampling points for the output
    output_positions = noise_output * torch.randn(
        n_samples, n_output_positions, n_sensors * n_measurements, n_dimensions
    )
    output_positions = (output_positions + measurements).reshape(
        n_samples, 1, n_output_positions * n_sensors * n_measurements, n_dimensions
    )

In [ ]:
# evaluate naive strategy of taking mean of measurements
input_mean = measurements.mean(2, True)
(input_mean - targets).abs().mean()

### Visualize the distribution of measurements and targets

In [ ]:
plt.figure()
for i in range(10):
    idx = np.random.randint(0, n_samples)
    plt.plot(*targets[idx, 0, 0].numpy(), f"C{i}x", markersize=10)
    plt.plot(*measurements[idx, 0].T.numpy(), f"C{i}.", markersize=5)
    plt.plot(*output_positions[idx, 0].T.numpy(), f"k.", markersize=1, alpha=0.05)
    plt.xlim(-width / 2, width / 2)
    plt.ylim(-width / 2, width / 2)
plt.show()

In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
from torchcps.kernel.rkhs import GaussianKernel
from torchcps.kernel.nn import (
    KernelConv,
    KernelMap,
    KernelNorm,
    KernelSample,
    Mixture,
)

max_filter_kernels = 128
n_channels = 1
n_weights = 32
in_weights = 1
out_weights = 1
update_positions = False

sigma = [5.0] * 3


class Model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.n_layers = len(sigma)
        self.nonlinearity = KernelMap(nn.LeakyReLU())
        self.readin = KernelMap(nn.Linear(in_weights, n_weights))
        self.readout = KernelMap(nn.Linear(n_weights, out_weights))

        conv_layers = []
        norm_layers = []
        linear_layers = []
        sample_layers = []
        for l in range(self.n_layers):
            first_layer = l == 0
            last_layer = l == self.n_layers - 1
            conv_layers += [
                KernelConv(
                    max_filter_kernels=max_filter_kernels,
                    in_channels=1 if first_layer else n_channels,
                    out_channels=1 if last_layer else n_channels,
                    n_dimensions=2,
                    kernel_spread=3 * sigma[l] * max_filter_kernels**0.5,
                    n_weights=n_weights,
                    update_positions=update_positions,
                    kernel_init="uniform",
                )
            ]
            linear_layers += [KernelMap(nn.Linear(n_weights, n_weights))]
            norm_layers += [KernelNorm(1 if last_layer else n_channels, n_weights)]
            sample_layers += [
                KernelSample(
                    kernel=GaussianKernel(sigma[l]),
                    alpha=None,
                    nonlinearity=nn.LeakyReLU(),
                )
            ]

        self.conv_layers = nn.ModuleList(conv_layers)
        self.norm_layers = nn.ModuleList(norm_layers)
        self.linear_layers = nn.ModuleList(linear_layers)
        self.sample_layers = nn.ModuleList(sample_layers)

    def forward(
        self,
        input: Mixture,
        output_positions: torch.Tensor,
    ):
        input_positions = input.positions.expand(-1, n_channels, -1, -1).contiguous()
        x: Mixture = self.readin(input)
        # x = self.nonlinearity(x)
        for l in range(self.n_layers):
            h = x.weights
            x = self.conv_layers[l](x)
            # sample on input positions for all layers except the last
            x = Mixture(x.positions.contiguous(), x.weights.contiguous())
            x = self.sample_layers[l](x, output_positions)
            x = self.linear_layers[l](x)
            # x = self.norm_layers[l](x)
            x = self.nonlinearity(x)
            # residual connection
            if l > 0:
                x = Mixture(x.positions, x.weights + h)

        x = self.readout(x)
        x = self.nonlinearity(x)
        return x


dataset = TensorDataset(measurements.cuda(), targets.cuda(), output_positions.cuda())
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)
model = Model().cuda()


# positions should not be regularized
parameters = list(model.named_parameters())
positions = [p for n, p in parameters if "_positions" in n]
weights = [p for n, p in parameters if "_positions" not in n]
optimizer = AdamW(
    [
        dict(params=weights, lr=1e-2, weight_decay=0.0),
        dict(params=positions, lr=1e-5, weight_decay=0.0),
    ],
)
# lr scheduling
scheduler = ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.1,
    patience=2,
)

mse_values = []
x_weights, y_weights = None, None
for i in range(20):
    pbar = tqdm(dataloader, total=len(dataloader))
    total_mse = 0.0
    for i, (x, y, z_pos) in enumerate(pbar):
        x_weights = torch.ones(*x.shape[:-1], device="cuda", requires_grad=False)[
            ..., None
        ]
        y_weights = torch.ones(*y.shape[:-1], device="cuda", requires_grad=False)[
            ..., None
        ]
        (z, z_weights) = model.forward(Mixture(x, x_weights), z_pos)

        mse = (
            GaussianKernel(sigma[-1])
            .squared_error(y, y_weights, z, z_weights)
            .mean(0)
            .sum()
        )
        total_mse += mse.item()
        mse_values.append(mse.detach().item())

        pbar.set_postfix(
            iter_mse=mse.item(),
            epoch_mse=total_mse / (i + 1),
            lr=optimizer.param_groups[0]["lr"],
        )

        optimizer.zero_grad()
        mse.backward()
        optimizer.step()
    scheduler.step(total_mse)

In [ ]:
print(np.mean(mse_values[-10:]))

plt.figure()
plt.plot(mse_values, label="MSE")
plt.legend()
plt.ylim(0, 2)
plt.show()

In [ ]:
def raster_rkhs(X: Mixture, sigma: float, width: float, resolution: int, relu=False):
    XY = (
        torch.stack(
            torch.meshgrid(
                torch.linspace(-width / 2, width / 2, resolution),
                torch.linspace(-width / 2, width / 2, resolution),
            ),
            dim=-1,
        )
        .reshape(-1, 2)
        .to(X.positions.device)
    )
    kernel = GaussianKernel(sigma)
    values = kernel(XY, X.positions[0, 0]) @ X.weights
    XY = XY.reshape(resolution, resolution, 2).detach()
    values = values.reshape(resolution, resolution).detach()
    return values, XY

In [ ]:
n_test = 100
indices = np.random.choice(n_samples, n_test, replace=False)

model.eval()
mae_mean = 0
mae_mode = 0
with torch.no_grad():
    for idx in tqdm(indices):
        x = measurements[None, idx, ...].cuda()
        y = targets[None, idx, None, :].cuda()
        out = output_positions[None, idx, ...].cuda()
        x_weights = torch.ones(*x.shape[:-1], device=x.device)[..., None]
        y_weights = torch.ones(*y.shape[:-1], device=y.device)[..., None]

        Z = model(Mixture(x, x_weights), out)

        # output argmax
        values, XY = raster_rkhs(Z, sigma[-1], width, 1000)
        # values.relu_()
        # expectation
        mean_xy = (XY * values[..., None]).sum((0, 1)) / values.sum()
        mae_mean += ((mean_xy - y.squeeze()).abs()).sum() / n_test
        # mode
        mode_xy = XY.reshape(-1, 2)[torch.argmax(values)]
        mae_mode += ((mode_xy - y.squeeze()).abs()).sum() / n_test
print(f"Mean Absolute Error (MEAN): {mae_mean.item():.2f}")
print(f"Mean Absolute Error (MODE): {mae_mode.item():.2f}")

In [ ]:
idx = np.random.randint(0, n_samples)
resolution = 2000


with torch.no_grad():
    x = measurements[None, idx, ...].cuda()
    y = targets[None, idx, None, :].cuda()
    x_weights = torch.ones(*x.shape[:-1], device=x.device)[..., None]
    y_weights = torch.ones(*y.shape[:-1], device=y.device)[..., None]
    out = output_positions[None, idx, ...].cuda()

    model.eval()
    with torch.no_grad():
        z, z_weights = model(Mixture(x, x_weights), out)
    model.train()

    # squeeze all the tensors
    x = x.squeeze().cpu()
    y = y.squeeze().cpu()

    extent = [-width / 2, width / 2, -width / 2, width / 2]
    values, XY = raster_rkhs(Mixture(z, z_weights), sigma[-1], width, resolution)
    # values = values.relu()
    # naive way to make predictions
    input_mean = x.mean(0)
    # expectation
    mean_xy = (XY * values[..., None]).sum((0, 1))
    # mode
    mode_xy = XY.reshape(-1, 2)[torch.argmax(values)]

    # get the intensity value at the target position
    target_value = GaussianKernel(sigma[-1])(y[None, None, None, :].cuda(), z)


plt.figure()
plt.imshow(values.T.cpu().detach(), extent=extent, origin="lower")

plt.plot(*x.T, ".", label="Measurements")
plt.plot(*y, "x", label="Target")
plt.plot(*input_mean, "o", label="Mean of Measurements")
plt.plot(*mean_xy.detach().cpu(), "o", label="Mean of CNN output")
plt.plot(*mode_xy.detach().cpu(), "o", label="Mode of CNN output")

# plt.xlim(extent[0], extent[1])
# plt.ylim(extent[2], extent[3])
# axis limits y +- w
w = 100
plt.xlim(y[0] - w, y[0] + w)
plt.ylim(y[1] - w, y[1] + w)

plt.legend()
plt.colorbar()
plt.show()

In [ ]:
def raster_filter(
    X: Mixture,
    sigma: float,
    weight_idx: int,
    channel: tuple[int, int],
    width: float,
    resolution: int,
):
    X_ = Mixture(
        X.positions[channel[0], channel[1], None, None, ...],
        X.weights[channel[0], channel[1], None, None, :, weight_idx, None].contiguous(),
    )
    return raster_rkhs(X_, sigma, width, resolution)[0].detach().cpu()


# Plot the CNN Filter at the lth layer
l = 0
channel = (0, 0)


conv_layers = model.conv_layers
positions = conv_layers[l].kernel_positions
weights = conv_layers[l].kernel_weights
assert isinstance(positions, torch.Tensor) and isinstance(weights, torch.Tensor)

n_weights = weights.shape[3]
fig, axs = plt.subplots(
    1,
    n_weights,
    figsize=(n_weights, 1),
    sharex=True,
    sharey=True,
)
for i in range(n_weights):
    rkhs = Mixture(positions, weights)
    filter_width = sigma[l] * 10
    extent = np.array([-1, 1, -1, 1]) * filter_width / 2

    if n_weights > 1:
        ax = axs[i]
    else:
        ax = axs

    ax.set_xticks([])
    ax.set_yticks([])

    ax.imshow(
        raster_filter(rkhs, sigma[l], i, channel, width=filter_width, resolution=32),
        extent=extent,
        origin="lower",
    )
plt.show()